In [11]:
import glob, re
import os
import warnings
import pprint
import json
import pandas as pd
from bs4 import BeautifulSoup
warnings.filterwarnings(action='ignore', category=UserWarning, module='bs4')

path = os.getcwd() + "/Annee2006/01/"
print(path)
#"Annee2011/01/"

/Users/yuyanq/Desktop/Bureau/M2/UE3 Dictionnaire et néologisme/projet-néologisme/2006-2011/Annee2006/01/


In [5]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [9]:
def read(path_fichier):
    with open(path_fichier, encoding = "utf8") as f:
        html_doc = f.read()
    return BeautifulSoup(html_doc, 'html.parser')

def enlever_nombres(texte):
    texte = re.sub("[0-9]+|[0-9]+[,\.-][0-9]+", "", texte)
    texte = re.sub("\'\'", "",  texte)
    return texte

def update_vocabulaire(set_mots, vocabulaire):
    dejavu = vocabulaire.intersection(set_mots)
    nouveau = set_mots.difference(vocabulaire)
    for x in set_mots:
        vocabulaire.add(x) # les types du corpus jusqu'à ce texte-là
    return vocabulaire, dejavu, nouveau

In [6]:
def isupper_lower(mots):
    mots_captalise= [mot for mot in mots if mot[0].isupper()==True]
    return len(mots_captalise)

def get_effectifs (liste_mots):                        
    dic_longueurs = {} 
    for mot in liste_mots:
        longueurs = len(mot)
        if longueurs not in dic_longueurs:
            dic_longueurs[longueurs] = 0
        dic_longueurs[longueurs]+= 1 
    return dic_longueurs 
                   
def get_taille(dic_longueurs):
    liste_effectifs = []
    for lon in range(0,31):
        if lon in dic_longueurs:
            liste_effectifs.append(dic_longueurs[lon])
        else:
            liste_effectifs.append(0)         
    return liste_effectifs   

In [7]:
def memoire_friendly(dico, path):
    try:
        with open(path, 'r') as f:
            old_dict = json.load(f)
    except:
        old_dict = {}

    new_dict = {**old_dict, **dico}
    for key in new_dict:
        new_dict[key] = list(new_dict[key])

    with open(path, 'w') as f:
        f.write(json.dumps(new_dict, indent=4, ensure_ascii=False))


def read_json_file(file_path):
    with open(file_path,encoding='utf8') as f:
        data = json.load(f)
    return data

## Test d'échantillon

In [12]:
path_fichier = path+"02_01_2006.xml"
soup = read(path_fichier)
#Créons un vocabulaire tiré de ce fichier du texte
L = []
for d in soup.find_all("div", type="article"):
    texte = d.text
    texte = enlever_nombres(texte)
    mots = re.findall("([A-Z][A-Z0-9.]+|[cCdDjJlLmnNsSt]'|[qQ]u'|\w+[\w'-]+\w+|\w\w+)", texte) # regex sans \S et sans chiffre
    L+=mots

In [13]:
print(sorted(set(L), reverse=True)[:10])

['ütre', 'ôter', 'îlot', 'îles', 'île', 'ëL', 'êtres', 'être', 'êtes', 'événements']


## Présentation du corpus et de la dictionnaire Glaff

###préparons notre vocabualire issu du glaff
import re, json
import glob

with open("freq_glaff_%i.json"%cpt, "w",encoding = "utf8") as w:
    w.write(json.dumps(dic_glaff,indent = 4, ensure_ascii = False)) #lisible pour l'humian
set_mots_glaff = set(dic_glaff.keys())
print("Taille Glaff : %i types"%len(set_mots_glaff))  # type(mot unique) VS token
#print("Entrée Glaff : %i formes"%(cpt)) 

In [14]:
with open("../freq_glaff_1406857.json", encoding="utf8") as t:
    dic_glaff = json.load(t)

set_mots_glaff = set(dic_glaff.keys())
print("Taille Glaff : %i types"%len(set_mots_glaff)) 

Taille Glaff : 1082688 types


## Traitement du corpus 
- tokeniser
- sauvegarder les tokens par jour 

In [16]:
import re
import datetime
articles_par_date = {}

for path_fichier in glob.glob("Annee2006/*/*"):
    
    date = (path_fichier.split("/")[-1])[:-4]
    #print(date)
    soup = read(path_fichier)
    liste_mots_par_jour =[]
    for cpt, d in enumerate(soup.find_all("div", type="article")):
        texte = d.text
        texte = enlever_nombres(texte)
        mots = re.findall("([A-Z][A-Z0-9.]+|[cCdDjJlLmnNsSt]'|[qQ]u'|\w+[\w'-]+\w+|\w\w+)", texte) #[\w'-]+
        #(\w+['-]\w+)+|\w+)
        # \w+['-]\w+ : les mots composés
        liste_mots_par_jour+=mots
        #save the set of words by day
    if date in articles_par_date:
        articles_par_date[date] = articles_par_date[date].union(set(liste_mots_par_jour))
    else:
        articles_par_date[date] = set(liste_mots_par_jour)

In [17]:
chemin_mots_Annee="./mot_par_jour/mot_articles_par_Annee2006.json"
memoire_friendly(articles_par_date, chemin_mots_Annee)
articles_par_date = read_json_file(chemin_mots_Annee)
print("%s a une collection de %s jours"%(chemin_mots_Annee[-14:-5],len(articles_par_date)))
print("Le premier jour est le %s et le dernier jour est le %s"%(sorted(articles_par_date.keys())[0],sorted(articles_par_date.keys())[-1]))

Annee2006 a une collection de 357 jours
Le premier jour est le 01_02_2006 et le dernier jour est le 31_12_2006


In [18]:
#comparer avec le glaff, calculer la proportion de smots qui sont absents du glaff

vocabulaire_glaff_update = set(dic_glaff.keys())
neologismeMIN = set()
proportion_mots_nouveauxMIN = []
liste_effectifs = []

sorted_date = sorted(articles_par_date.keys(), key=lambda x: datetime.datetime.strptime(x, '%d_%m_%Y'))

for date in sorted_date:
    #print(date)

    set_mots_jour = set(articles_par_date[date])
    vocabulaire_glaff_update, dejavu, nouveau = update_vocabulaire(set_mots_jour, vocabulaire_glaff_update)
    # cas 2 + cas 3 # nouveau
    mots_dans_glaff = set(set_mots_jour).intersection(set(dic_glaff.keys())) # cas 1 + cas 4

    ##cas 1:min + glaff ##cas 4:maj + glaff
    Lower_mots_dans_glaff = [mot.lower()for mot in mots_dans_glaff] # min+maj glaff
    cas1_min_glaff = mots_dans_glaff.intersection(Lower_mots_dans_glaff)
    cas4_maj_glaff = mots_dans_glaff.difference(Lower_mots_dans_glaff)

    ##cas 2:min + pas glaff ##cas 3:maj + pas glaff
    Lower_mots_abscent = [mot.lower()for mot in nouveau] # min+maj glaff
    cas2_min_non_glaff =  nouveau.intersection(Lower_mots_abscent)
    cas3_maj_non_glaff = nouveau.difference(Lower_mots_abscent)
   
    neologismeMIN = neologismeMIN.union(cas2_min_non_glaff)
    liste_jour = [date,len(nouveau),len(cas2_min_non_glaff)]
    
    proportion_mots_nouveauxMIN.append((len(cas2_min_non_glaff)/len(set_mots_jour)))
    liste_effectifs.append(liste_jour)
print("-"*50)
print("NB de mots nouveaux minisculs en total :",len(neologismeMIN))

--------------------------------------------------
NB de mots nouveaux minisculs en total : 72065


In [20]:
chemin_neo_Annee="/Users/yuyanq/Desktop/Bureau/M2/UE3 Dictionnaire et néologisme/projet-néologisme/1999-2003/neo/neo.json"

neologismeDico = {"Annee2006": neologismeMIN}
memoire_friendly(neologismeDico, chemin_neo_Annee)
print("%s a une collection de %s néologismes"%("Annee2006",len(neologismeMIN)))

Annee2006 a une collection de 72065 néologismes


### Webster's Unabridged English Dictionary.

In [21]:
###préparons notre vocabualire anglaise
# https://github.com/adambom/dictionary


with open("../dictionary.json", encoding="utf8") as t:
    dic_Webster = json.load(t)

set_mots_Webster = set([mot.lower() for mot in dic_Webster.keys()])
print("Taille Webster : %i types"%len(set_mots_Webster))  # type(mot unique) VS token

neo_emprunt = list(set(neologismeMIN).intersection(set_mots_Webster))
print("Taille neo : %i types"%len(neo_emprunt))  # type(mot unique) VS token

Taille Webster : 86036 types
Taille neo : 1980 types


In [22]:
chemin_neo_emprunt_Annee="/Users/yuyanq/Desktop/Bureau/M2/UE3 Dictionnaire et néologisme/projet-néologisme/1999-2003//neo_emprunt/neo_emprunt.json"
neologisme_enprunt_Dico = {"Annee2006": neo_emprunt}
memoire_friendly(neologisme_enprunt_Dico, chemin_neo_emprunt_Annee)